In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

import pystan
import pystan.chains
from collections import OrderedDict
import pandas as pd

plt.style.use('seaborn-darkgrid')
print('Runing on PyMC3 v{}'.format(pm.__version__))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Runing on PyMC3 v3.3


In [2]:
f1 = '/usr/local/lib/python3.5/dist-packages/pystan/tests/data/blocker.1.csv'
f2 = '/usr/local/lib/python3.5/dist-packages/pystan/tests/data/blocker.2.csv'

# f1 = '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pystan/tests/data/blocker.1.csv'
# f2 = '/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pystan/tests/data/blocker.2.csv'

# read csv using numpy
c1 = np.loadtxt(f1, skiprows=41, delimiter=',')[:, 4:]
c1_colnames = open(f1, 'r').readlines()[36].strip().split(',')[4:]
np.testing.assert_equal(c1_colnames[0], 'd')
c2 = np.loadtxt(f2, skiprows=41, delimiter=',')[:, 4:]
c2_colnames = open(f2, 'r').readlines()[36].strip().split(',')[4:]
np.testing.assert_equal(c1_colnames, c2_colnames)
np.testing.assert_equal(len(c1_colnames), c1.shape[1])

n_samples = len(c1)
np.testing.assert_equal(n_samples, 1000)

c1 = OrderedDict((k, v) for k, v in zip(c1_colnames, c1.T))
c2 = OrderedDict((k, v) for k, v in zip(c2_colnames, c2.T))

lst = dict(fnames_oi=c1_colnames, samples=[{'chains': c1}, {'chains': c2}],
           n_save=np.repeat(n_samples, 2), permutation=None,
           warmup=0, warmup2=[0, 0], chains=2, n_flatnames=len(c1))

In [3]:
slst = lst['samples'][0]['chains']
slst.keys()

odict_keys(['d', 'sigmasq_delta', 'mu.1', 'mu.2', 'mu.3', 'mu.4', 'mu.5', 'mu.6', 'mu.7', 'mu.8', 'mu.9', 'mu.10', 'mu.11', 'mu.12', 'mu.13', 'mu.14', 'mu.15', 'mu.16', 'mu.17', 'mu.18', 'mu.19', 'mu.20', 'mu.21', 'mu.22', 'delta.1', 'delta.2', 'delta.3', 'delta.4', 'delta.5', 'delta.6', 'delta.7', 'delta.8', 'delta.9', 'delta.10', 'delta.11', 'delta.12', 'delta.13', 'delta.14', 'delta.15', 'delta.16', 'delta.17', 'delta.18', 'delta.19', 'delta.20', 'delta.21', 'delta.22', 'delta_new', 'sigma_delta'])

In [4]:
param_names = list(slst.keys())

In [5]:
m = lst['chains']

ns_save = lst['n_save']
ns_warmup2 = lst['warmup2']
ns_kept = [s - w for s, w in zip(lst['n_save'], lst['warmup2'])]

n_samples = min(ns_kept)

In [6]:
n_eff = [
    466.099, 136.953, 1170.390, 541.256,
    518.051, 589.244, 764.813, 688.294,
    323.777, 502.892, 353.823, 588.142,
    654.336, 480.914, 176.978, 182.649,
    642.389, 470.949, 561.947, 581.187,
    446.389, 397.641, 338.511, 678.772,
    1442.250, 837.956, 869.865, 951.124,
    619.336, 875.805, 233.260, 786.568,
    910.144, 231.582, 907.666, 747.347,
    720.660, 195.195, 944.547, 767.271,
    723.665, 1077.030, 470.903, 954.924,
    497.338, 583.539, 697.204, 98.421
]

ess = []
for i in range(len(n_eff)):
    ess.append(pystan.chains.ess(lst, i))
    np.testing.assert_almost_equal(ess[i], n_eff[i], 2)

In [7]:
from pymc3.util import get_default_varnames
from pymc3.backends.base import MultiTrace

In [8]:
from scipy.signal import fftconvolve


def autocorr(x, lag=None):
    """
    Compute autocorrelation using FFT for every lag for the input array
    https://en.wikipedia.org/wiki/Autocorrelation#Efficient_computation

    Parameters
    ----------
    x : Numpy array
        An array containing MCMC samples

    Returns
    -------
    acorr: Numpy array same size as the input array
    """
    y = x - x.mean()
    n = len(y)
    result = fftconvolve(y, y[::-1])
    acorr = result[len(result) // 2:]
    acorr /= np.arange(n, 0, -1)
    acorr /= acorr[0]
    if lag is None:
        return acorr
    else:
        warnings.warn(
            "The `lag` argument has been deprecated. If you want to get "
            "the value of a specific lag please call `autocorr(x)[lag]`.",
            DeprecationWarning)
        return acorr[lag]


def autocov(x, lag=None):
    """Compute autocovariance estimates for every lag for the input array

    Parameters
    ----------
    x : Numpy array
        An array containing MCMC samples

    Returns
    -------
    acov: Numpy array same size as the input array
    """
    acorr = autocorr(x)
    varx = np.var(x, ddof=1) * (len(x) - 1) / len(x)
    acov = acorr * varx
    if lag is None:
        return acov
    else:
        warnings.warn(
            "The `lag` argument has been deprecated. If you want to get "
            "the value of a specific lag please call `autocov(x)[lag]`.",
            DeprecationWarning)
        return acov[lag]

In [60]:
def effective_n(mtrace, varnames=None, include_transformed=False):
    R"""Returns estimate of the effective sample size of a set of traces.

    Parameters
    ----------
    mtrace : MultiTrace or trace object
      A MultiTrace object containing parallel traces (minimum 2)
      of one or more stochastic parameters.
    varnames : list
      Names of variables to include in the effective_n report
    include_transformed : bool
      Flag for reporting automatically transformed variables in addition
      to original variables (defaults to False).

    Returns
    -------
    n_eff : dictionary of floats (MultiTrace) or float (trace object)
        Return the effective sample size, :math:`\hat{n}_{eff}`

    Notes
    -----
    The diagnostic is computed by:

    .. math:: \hat{n}_{eff} = \frac{mn}{1 + 2 \sum_{t=1}^T \hat{\rho}_t}

    where :math:`\hat{\rho}_t` is the estimated autocorrelation at lag t, and T
    is the first odd positive integer for which the sum
    :math:`\hat{\rho}_{T+1} + \hat{\rho}_{T+1}` is negative.

    The current implementation is similar to Stan, which uses Geyer's initial
    monotone sequence criterion (Geyer, 1992; Geyer, 2011).

    References
    ----------
    Gelman et al. BDA (2014)"""

    def get_neff(x):
        """Compute the effective sample size for a 2D array
        """
        trace_value = x.T
        nchain, n_samples = trace_value.shape

        acov = np.asarray([autocov(trace_value[chain])
                           for chain in range(nchain)])

        chain_mean = trace_value.mean(axis=1)
        chain_var = acov[:, 0] * n_samples / (n_samples - 1.)
        acov_t = acov[:, 1] * n_samples / (n_samples - 1.)
        mean_var = np.mean(chain_var)
        var_plus = mean_var * (n_samples - 1.) / n_samples
        var_plus += np.var(chain_mean, ddof=1)

        rho_hat_t = np.zeros(n_samples)
        rho_hat_even = 1.
        rho_hat_t[0] = rho_hat_even
        rho_hat_odd = 1. - (mean_var - np.mean(acov_t)) / var_plus
        rho_hat_t[1] = rho_hat_odd
        # Geyer's initial positive sequence
        max_t = 1
        t = 1
        while t < (n_samples - 2) and (rho_hat_even + rho_hat_odd) >= 0.:
            rho_hat_even = 1. - (mean_var - np.mean(acov[:, t + 1])) / var_plus
            rho_hat_odd = 1. - (mean_var - np.mean(acov[:, t + 2])) / var_plus
            if (rho_hat_even + rho_hat_odd) >= 0:
                rho_hat_t[t + 1] = rho_hat_even
                rho_hat_t[t + 2] = rho_hat_odd
            max_t = t + 2
            t += 2

        # Geyer's initial monotone sequence
        t = 3
        while t <= max_t - 2:
            if (rho_hat_t[t + 1] + rho_hat_t[t + 2]) > (rho_hat_t[t - 1] + rho_hat_t[t]):
                rho_hat_t[t + 1] = (rho_hat_t[t - 1] + rho_hat_t[t]) / 2.
                rho_hat_t[t + 2] = rho_hat_t[t + 1]
            t += 2
        ess = nchain * n_samples
        ess = ess / (-1. + 2. * np.sum(rho_hat_t))
        return ess

    def generate_neff(trace_values):
        x = np.array(trace_values)
        shape = x.shape

        # Make sure to handle scalars correctly, adding extra dimensions if
        # needed. We could use np.squeeze here, but we don't want to squeeze
        # out dummy dimensions that a user inputs.
        if len(shape) == 2:
            x = np.atleast_3d(trace_values)

        # Transpose all dimensions, which makes the loop below
        # easier by moving the axes of the variable to the front instead
        # of the chain and sample axes.
        x = x.transpose()

        # Get an array the same shape as the var
        _n_eff = np.zeros(x.shape[:-2])

        # Iterate over tuples of indices of the shape of var
        for tup in np.ndindex(*list(x.shape[:-2])):
            _n_eff[tup] = get_neff(x[tup])

        if len(shape) == 2:
            return _n_eff[0]

        return np.transpose(_n_eff)

    if not isinstance(mtrace, MultiTrace):
        # Return neff for non-multitrace array
        return generate_neff(mtrace)

    if mtrace.nchains < 2:
        raise ValueError(
            'Calculation of effective sample size requires multiple chains '
            'of the same length.')

    if varnames is None:
        varnames = get_default_varnames(
            mtrace.varnames, include_transformed=include_transformed)

    n_eff = {}

    for var in varnames:
        n_eff[var] = generate_neff(mtrace.get_values(var, combine=False))

    return n_eff

In [64]:
ess2 = []
for i, varname in enumerate(param_names):
    trace_values = [lst['samples'][im]['chains'][varname] for im in range(m)]
    ess2.append(effective_n(trace_values))
    np.testing.assert_almost_equal(ess2[i], n_eff[i], 0)

In [58]:
df_neff = pd.DataFrame(data=dict(Target=n_eff, PyStan=ess, PyMC3=ess2),
                       columns=['Target', 'PyStan', 'PyMC3'])
df_neff

,Target,PyStan,PyMC3
0,466.099,466.098810,465.994306
1,136.953,136.952532,136.941283
2,1170.390,1170.393732,1170.222485
3,541.256,541.255659,541.199238
4,518.051,518.051325,517.996363
5,589.244,589.243546,589.163026
6,764.813,764.812721,764.667763
7,688.294,688.293542,688.212347
8,323.777,323.777181,323.743779
9,502.892,502.891905,502.837470


In [14]:
%timeit pm.effective_n(trace_values)

960 µs ± 7.29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit effective_n(trace_values)

776 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [106]:
trace_values2 = [np.random.randn(10000) for im in range(20)]

In [107]:
%timeit pm.effective_n(trace_values2)

1.99 ms ± 57.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [108]:
%timeit effective_n(trace_values2)

22.6 ms ± 867 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [109]:
%timeit effective_n2(trace_values2)

21.9 ms ± 812 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
ess2 = []
for i, varname in enumerate(param_names):
    trace_values = [lst['samples'][im]['chains'][varname] for im in range(m)]
    ess2.append(effective_n2(trace_values))
    np.testing.assert_almost_equal(ess2[i], n_eff[i], 0)

In [ ]:
import numba

In [ ]:
@numba.jit(nopython=True)
def mult_conj(a):
    """Multiply a with conj(a) and store in a.
    
    [Re, Re, Im, Re, Im...]
    like output from scipy.fftpack.rfft
    """
    n = len(a)
    if n == 0:
        return
    a[0] = a[0] * a[0]
    for i in range(1, (n + 1) // 2):
        j, k = 2 * i - 1, 2 * i
        a[j] = a[j] * a[j] + a[k] * a[k]
        a[k] = 0
    if n > 1 and n % 2 == 0:
        a[n - 1] = a[n - 1] * a[n - 1]

        
@numba.jit
def autocov2(x):
    assert len(x.shape) == 2
    y = x - np.mean(x, axis=-1, keepdims=True)
    n = y.shape[-1]
    shape = 2 * n - 1
    
    fft_len = fftpack.next_fast_len(shape)
    fx = fftpack.rfft(y, fft_len, overwrite_x=False)
    for i in range(x.shape[0]):
        mult_conj(fx[i, :])
    out = fftpack.irfft(fx, fft_len, overwrite_x=False)
    out = out[..., :n]
    out[:] /= np.arange(n, 0, -1)
    return out


@numba.jit(nopython=True)
def get_neff_inner(trace_value, acov):
    """Compute the effective sample size for a 2D array
    """
    n_chains, n_samples = trace_value.shape
    assert acov.shape == (n_chains, n_samples)

    chain_mean = np.sum(trace_value, 1) / n_samples

    chain_var = acov[:, 0] * n_samples / (n_samples - 1.)
    acov_t = acov[:, 1] * n_samples / (n_samples - 1.)
    mean_var = np.mean(chain_var)
    var_plus = mean_var * (n_samples - 1.) / n_samples
    var_plus += np.var(chain_mean) * n_chains / (n_chains - 1.)

    rho_hat_t = np.zeros(n_samples)
    rho_hat_even = 1.
    rho_hat_t[0] = rho_hat_even
    rho_hat_odd = 1. - (mean_var - np.mean(acov_t)) / var_plus
    rho_hat_t[1] = rho_hat_odd
    max_t = 1
    t = 1

    acov_means = np.sum(acov, 0) / n_chains
    while t < (n_samples - 2) and (rho_hat_even + rho_hat_odd) >= 0.:
        rho_hat_even = 1. - (mean_var - acov_means[t + 1]) / var_plus
        rho_hat_odd = 1. - (mean_var - acov_means[t + 2]) / var_plus
        if (rho_hat_even + rho_hat_odd) >= 0:
            rho_hat_t[t + 1] = rho_hat_even
            rho_hat_t[t + 2] = rho_hat_odd
        max_t = t + 2
        t += 2

    t = 3
    while t <= max_t - 2:
        if (rho_hat_t[t + 1] + rho_hat_t[t + 2]) > (rho_hat_t[t - 1] + rho_hat_t[t]):
            rho_hat_t[t + 1] = (rho_hat_t[t - 1] + rho_hat_t[t]) / 2.
            rho_hat_t[t + 2] = rho_hat_t[t + 1]
        t += 2
    ess = nchain * n_samples
    ess = ess / (-1. + 2. * np.sum(rho_hat_t))
    return ess


@numba.jit()
def get_neff2(x):
    """Compute the effective sample size for a 2D array
    """
    neff = np.zeros(x.shape[0])
    nvals, nchain, n_samples = x.shape
    
    for i in range(len(x)):
        trace_value = x[i]
        nchain, n_samples = trace_value.shape
        acov = autocov2(trace_value)
        neff[i] = get_neff_inner(trace_value, acov)
    return neff


def effective_n2(mtrace, varnames=None, include_transformed=False):
    def generate_neff(trace_values):
        """Trace values is an array (*varshape, n_chains, n_samples)"""
        x = np.stack([val.T for val in trace_values], axis=-2)
        *varshape, n_chains, n_samples = x.shape
        x = np.ascontiguousarray(x.reshape((-1, n_chains, n_samples)))
        return get_neff2(x).reshape(varshape)

    if not isinstance(mtrace, pm.backends.base.MultiTrace):
        # Return neff for non-multitrace array
        return generate_neff(mtrace)

    if mtrace.nchains < 2:
        raise ValueError(
            'Calculation of effective sample size requires multiple chains '
            'of the same length.')

    if varnames is None:
        varnames = pm.util.get_default_varnames(mtrace.varnames,include_transformed=include_transformed)

    n_eff = {}

    for var in varnames:
        n_eff[var] = generate_neff(mtrace.get_values(var, combine=False))

    return n_eff

In [99]:
def effective_n2(mtrace, varnames=None, include_transformed=False):
    R"""Returns estimate of the effective sample size of a set of traces.

    Parameters
    ----------
    mtrace : MultiTrace or trace object
      A MultiTrace object containing parallel traces (minimum 2)
      of one or more stochastic parameters.
    varnames : list
      Names of variables to include in the effective_n report
    include_transformed : bool
      Flag for reporting automatically transformed variables in addition
      to original variables (defaults to False).

    Returns
    -------
    n_eff : dictionary of floats (MultiTrace) or float (trace object)
        Return the effective sample size, :math:`\hat{n}_{eff}`

    Notes
    -----
    The diagnostic is computed by:

    .. math:: \hat{n}_{eff} = \frac{mn}{1 + 2 \sum_{t=1}^T \hat{\rho}_t}

    where :math:`\hat{\rho}_t` is the estimated autocorrelation at lag t, and T
    is the first odd positive integer for which the sum
    :math:`\hat{\rho}_{T+1} + \hat{\rho}_{T+1}` is negative.

    The current implementation is similar to Stan, which uses Geyer's initial
    monotone sequence criterion (Geyer, 1992; Geyer, 2011).

    References
    ----------
    Gelman et al. BDA (2014)"""

    def get_neff(x):
        """Compute the effective sample size for a 2D array
        """
        trace_value = x.T
        nchain, n_samples = trace_value.shape

        acov = np.asarray([autocov(trace_value[chain])
                           for chain in range(nchain)])

        chain_mean = trace_value.mean(axis=1)
        chain_var = acov[:, 0] * n_samples / (n_samples - 1.)
        acov_t = acov[:, 1] * n_samples / (n_samples - 1.)
        mean_var = np.mean(chain_var)
        var_plus = mean_var * (n_samples - 1.) / n_samples
        var_plus += np.var(chain_mean, ddof=1)

        rho_hat_t = np.zeros(n_samples)
        rho_hat_t[0] = 1.
        rho_hat_t[1] = 1. - (mean_var - np.mean(acov_t)) / var_plus
        # Geyer's initial positive sequence
        rho_hat_tmp = 1. - (mean_var - np.mean(acov, axis=0)) / var_plus
        t_ = np.where((rho_hat_tmp[2::2] + rho_hat_tmp[3::2]) < 0)[0][0]
        max_t = 2*t_+2
        rho_hat_t[2:max_t] = rho_hat_tmp[2:max_t]

        # Geyer's initial monotone sequence
        t = 3
        while t <= max_t - 2:
            if (rho_hat_t[t + 1] + rho_hat_t[t + 2]) > (rho_hat_t[t - 1] + rho_hat_t[t]):
                rho_hat_t[t + 1] = (rho_hat_t[t - 1] + rho_hat_t[t]) / 2.
                rho_hat_t[t + 2] = rho_hat_t[t + 1]
            t += 2
        ess = nchain * n_samples
        ess = ess / (-1. + 2. * np.sum(rho_hat_t))
        return ess

    def generate_neff(trace_values):
        x = np.array(trace_values)
        shape = x.shape

        # Make sure to handle scalars correctly, adding extra dimensions if
        # needed. We could use np.squeeze here, but we don't want to squeeze
        # out dummy dimensions that a user inputs.
        if len(shape) == 2:
            x = np.atleast_3d(trace_values)

        # Transpose all dimensions, which makes the loop below
        # easier by moving the axes of the variable to the front instead
        # of the chain and sample axes.
        x = x.transpose()

        # Get an array the same shape as the var
        _n_eff = np.zeros(x.shape[:-2])

        # Iterate over tuples of indices of the shape of var
        for tup in np.ndindex(*list(x.shape[:-2])):
            _n_eff[tup] = get_neff(x[tup])

        if len(shape) == 2:
            return _n_eff[0]

        return np.transpose(_n_eff)

    if not isinstance(mtrace, MultiTrace):
        # Return neff for non-multitrace array
        return generate_neff(mtrace)

    if mtrace.nchains < 2:
        raise ValueError(
            'Calculation of effective sample size requires multiple chains '
            'of the same length.')

    if varnames is None:
        varnames = get_default_varnames(
            mtrace.varnames, include_transformed=include_transformed)

    n_eff = {}

    for var in varnames:
        n_eff[var] = generate_neff(mtrace.get_values(var, combine=False))

    return n_eff